<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Automated_Retraining_Workflow_Example_with_MLflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mlflow

In [ ]:
import mlflow
import mlflow.pytorch
import torch

# Define your model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x):
        return self.fc(x)

# Instantiate the model
model = MyModel()

# Example input for the model
input_example = torch.randn(1, 10).numpy()  # Convert to numpy.ndarray

# Placeholder for model performance comparison
new_model_performance = 0.85  # Example performance metric (e.g., accuracy)
baseline_performance = 0.80  # Example baseline performance metric (e.g., accuracy)

# Define the conda environment
conda_env = {
    'name': 'mlflow-env',
    'channels': ['defaults', 'conda-forge'],
    'dependencies': [
        'python=3.8',
        'pytorch=2.5.1',
        'pip',
        {
            'pip': [
                f'torch=={torch.__version__}',
                'mlflow'
            ]
        }
    ]
}

# Log model and parameters
with mlflow.start_run():
    mlflow.log_params({"learning_rate": 0.01, "epochs": 10})
    mlflow.pytorch.log_model(model, "model", conda_env=conda_env, input_example=input_example)

    # Log new model performance
    mlflow.log_metric("new_model_performance", new_model_performance)

    # Evaluate performance and deploy if improved
    if new_model_performance > baseline_performance:
        print("New model performance improved. Ready to deploy.")
        # Remove existing directory if it exists
        import shutil
        shutil.rmtree("/content/production_model_path", ignore_errors=True)
        # Save the model to a production path
        mlflow.pytorch.save_model(model, "/content/production_model_path")